In [1]:
import xml.etree.ElementTree as ET
from pathlib import Path
from collections import Counter

import json

import pandas as pd


#from utils import *

In [2]:
import xml.etree.ElementTree as ET
from pathlib import Path
from collections import Counter, defaultdict
import json
import pandas as pd

CLASS_MAP = {
    "MAN_MADE_EVENT": "MANMADE_DISASTER",
    "NATURAL_EVENT": "NATURAL_DISASTER"
}

def transform_label(y):
    return pd.Series(y).str.split(".", expand=True)[0]

def transform_probs(probs, clf):
    clf_map = defaultdict(list)
    for c in clf.classes_:
        clf_map[c.split(".")[0]].append(c)
    df_probs = pd.DataFrame(probs, columns=clf.classes_)
    for c_parent, children in clf_map.items():
        df_probs[c_parent] = df_probs[children].sum(axis=1)
    return df_probs[list(clf_map)].idxmax(axis=1)

def get_w_children(node, base_node=None, split_paras=False):
    idx = 0
    if base_node is None:
        base_node = ET.Element('P')

    for child in node:
        if child.tag == "W":
            label = base_node.tag
            if base_node.attrib.get("TYPE"):
                label = f"{label}.{base_node.attrib['TYPE']}"
            label = f"B-{label}" if idx == 0 else f"I-{label}"
            if base_node.tag == "P":
                label = "O"
            token_text = (child.text or "").strip()
            if token_text:
                yield token_text, label
                idx += 1
        else:
            yield from get_w_children(child, base_node=child)
    if split_paras and node.tag == "P":
        yield "<P>", "P"

def get_w_children_test(node, base_node=None, split_paras=False):
    for child in node:
        if child.tag == "P":
            text_content = child.text or ""
            tokens = [t.strip() for t in text_content.split("  ") if t.strip()]
            for token in tokens:
                yield token, "O"
            if split_paras:
                yield "<P>", "P"

def split_tag(tag):
    return tuple(tag.split("-", 1)) if tag != "O" else (tag, None)

def extract_entities(tags):
    tags = list(tags)
    curr_entity = []
    entities = []
    for i, tag in enumerate(tags + ["O"]):
        boundary, label = split_tag(tag)
        if curr_entity:
            if boundary in {"B", "O"} or label != curr_entity[-1][1]:
                start = i - len(curr_entity)
                end = i
                entity_label = curr_entity[-1][1]
                entities.append((entity_label, start, end))
                curr_entity = []
            elif boundary == "I":
                curr_entity.append((boundary, label))
        if boundary == "B":
            assert not curr_entity, f"Entity should be empty. Found: {curr_entity}"
            curr_entity.append((boundary, label))
    return entities

def get_entity_info(bio_labels, tokens, text=None, spans=None):
    entities_info = extract_entities(bio_labels)
    entities = []
    for label, start, end in entities_info:
        entity_phrase = " ".join(tokens[start:end])
        entities.append(dict(
            tokens=tokens[start:end],
            label=label,
            start=start,
            end=end,
            entity_phrase=entity_phrase
        ))
    return entities

def xml_to_conll(xml_path, test=False):
    try:
        tree = ET.parse(xml_path)
        root = tree.getroot()
        parse_fn = get_w_children_test if test else get_w_children
        seq = list(parse_fn(root))
        return seq
    except Exception as e:
        print(f" Error parsing XML file {xml_path}: {e}")
        return []

def xml_to_json(xml_path, test=False):
    seq = xml_to_conll(xml_path, test=test)
    if not seq:
        print(f" Skipping file {xml_path} due to errors")
        return {"docid": xml_path.stem, "tokens": [], "tags": [], "labels": {}}

    docid = xml_path.stem
    tokens, tags = zip(*seq) if seq else ([], [])

    labels = Counter([t[2:] for t in tags if t.startswith(("B-MAN_MADE_EVENT", "B-NATURAL_EVENT"))])
    return {
        "docid": docid,
        "tokens": tokens,
        "tags": tags,
        "labels": labels
    }

import json

def process(xml_dir_path, test=False):
    import os
    from pathlib import Path
    import json

    xml_dir = Path(xml_dir_path)

    # Exact file name and path
    output_path = Path(r"C:\semisters\SEM6\NLP\F_project\data\processed\te\train.json")

    files = list(xml_dir.glob("*.xml"))
    if not files:
        print(f" No XML files found in {xml_dir}")
        return

    all_data = []
    for xml_file in files:
        json_data = xml_to_json(xml_file, test=test)
        all_data.append(json_data)

    # Make sure the target directory exists
    os.makedirs(output_path.parent, exist_ok=True)

    # Write to train.json
    with open(output_path, "w", encoding="utf-8") as f:
        for entry in all_data:
            f.write(json.dumps(entry, ensure_ascii=True) + "\n")

    print(f" Successfully saved output to {output_path}")




In [3]:
tree = ET.parse(r'C:\semisters\SEM6\NLP\F_project\data\raw\te\Train\file01.xml')
root = tree.getroot()

In [4]:
root

<Element 'DOCUMENT' at 0x000001EEBFB6DEF0>

In [5]:
list(get_w_children(root))  

[('జమ్మూవిల్లే', 'B-PLACE-ARG'),
 ('భారీ వర్షపాతం ద్వారా', 'B-REASON-ARG'),
 ('చాలా', 'O'),
 ('స్థానాలు', 'O'),
 ('కొండచరియలు', 'B-NATURAL_EVENT.LAND_SLIDE'),
 ('రవాణా', 'B-AFTER_EFFECTS-ARG'),
 ('కత్తిరించండి', 'I-AFTER_EFFECTS-ARG'),
 ('జమ్మూ', 'O'),
 ('జమ్మూ', 'O'),
 ('కాశ్మీర్', 'O'),
 ('రాష్ట్రంలో', 'O'),
 ('రోజులు', 'B-TIME-ARG'),
 ('భారీ వర్షం', 'O'),
 ('నియామకం', 'O'),
 ('పైకి వస్తుంది', 'O'),
 ('ప్రత్యేకంగా', 'O'),
 ('రాంబన్', 'B-PLACE-ARG'),
 ('ఉదంబూర్', 'I-PLACE-ARG'),
 ('జిల్లాల్లో', 'I-PLACE-ARG'),
 ('ఉధెం', 'B-REASON-ARG'),
 ('షవర్', 'I-REASON-ARG'),
 ('బ్లీచింగ్', 'I-REASON-ARG'),
 ('కొనుగోలు', 'I-REASON-ARG'),
 ('జిల్లాలు', 'O'),
 ('చాలా', 'O'),
 ('స్థానాలు', 'O'),
 ('కొండచరియలు', 'B-NATURAL_EVENT.LAND_SLIDE'),
 ('సంభవించింది', 'O'),
 ('కొండచరియలు ఉంటే', 'B-NATURAL_EVENT.LAND_SLIDE'),
 ('చాలా', 'O'),
 ('స్థానాలు', 'O'),
 ('రవాణా', 'B-AFTER_EFFECTS-ARG'),
 ('కత్తిరించబడింది', 'I-AFTER_EFFECTS-ARG'),
 ('గురించి', 'O'),
 ('రవాణా', 'O'),
 ('అధికారం', 'O'),
 ('ఒకటి', 'O'),
 

In [6]:
xml_path = Path(r'C:\semisters\SEM6\NLP\F_project\data\raw\te\Train\file501.xml')
json_data = xml_to_json(xml_path)
print(json_data)

{'docid': 'file501', 'tokens': ('వరదలు', 'సెర్బియా', 'మోడీ', 'ప్రభుత్వం', 'సహాయకారి', 'నిరీక్షణ', 'ఏదైనా', 'రోజు నుండి రోజుకు', 'వరదలు కారణంగా', 'వదిలివేసిన వారిలో', 'నష్టం', 'కారణం', 'సెర్బియా', 'దేశం', 'తిరిగి మూల్యాంకనం కోసం', 'భారతదేశం', 'కొత్తగా', 'స్థాపన', 'కారణం', 'మోడీ', 'ప్రభుత్వం', 'సహాయకారి', 'నిరీక్షణ', 'కెల్లీ', 'సెర్బియా', 'భారతదేశంలో', 'రాయబారి', 'డాన్', 'మిరిలోవిక్', 'సమాచారం', 'ఇవ్వబడింది', 'మిరిలోవిమ్', 'అన్నారు', 'సెర్బియా', 'మూడు', 'నెల', 'పతనం', 'కారిడార్', 'వర్షం', 'మూడు', 'రోజు', 'పడిపోతుంది', 'ఈ కారణంగా', 'దేశాలు', 'వదిలివేసిన వారిలో', 'నష్టం', 'జరిగింది', 'మే', 'మే', 'వరకు', 'దేశంలో', 'అత్యవసర పరిస్థితి', 'పబ్లిక్', 'వచ్చింది', 'కారిడార్', 'వర్షాలు', 'దేశం ముందు', 'గొప్పది', 'సంక్షోభం', 'సృష్టి', 'తయారు చేయబడింది', 'ఏమైనా', 'దేశం', 'అనేక', 'పాక్షిక', 'ఎక్స్ట్రీమ్', 'నష్టం', 'జరిగింది', 'దెబ్బతింది', 'పాక్షిక', 'వినోదం', 'చేయడానికి', 'అవసరం', 'తయారు చేయబడింది', 'దాని కోసం', 'మాకు', 'మోడీ', 'ప్రభుత్వం', 'సహాయకారి', 'నిరీక్షణ', 'భారతీయుడు', 'సహాయం ద్వారా', 'మేము',

In [7]:
get_entity_info(json_data["tags"], json_data["tokens"], text=None, spans=None)

[{'tokens': ('వరదలు',),
  'label': 'NATURAL_EVENT.FLOODS',
  'start': 0,
  'end': 1,
  'entity_phrase': 'వరదలు'},
 {'tokens': ('సెర్బియా',),
  'label': 'PLACE-ARG',
  'start': 1,
  'end': 2,
  'entity_phrase': 'సెర్బియా'},
 {'tokens': ('ఏదైనా', 'రోజు నుండి రోజుకు'),
  'label': 'TIME-ARG',
  'start': 6,
  'end': 8,
  'entity_phrase': 'ఏదైనా రోజు నుండి రోజుకు'},
 {'tokens': ('వరదలు కారణంగా',),
  'label': 'NATURAL_EVENT.FLOODS',
  'start': 8,
  'end': 9,
  'entity_phrase': 'వరదలు కారణంగా'},
 {'tokens': ('వదిలివేసిన వారిలో', 'నష్టం'),
  'label': 'AFTER_EFFECTS-ARG',
  'start': 9,
  'end': 11,
  'entity_phrase': 'వదిలివేసిన వారిలో నష్టం'}]

In [11]:

process(r"C:\Users\kotav\Desktop\NLP\data\raw\te\Train")

 Successfully saved output to C:\semisters\SEM6\NLP\F_project\data\processed\te\train.json
